# Sprint23 LSTM

## 【問題1】各種手法の実行

Kerasには4種類のReccurentレイヤーが用意されています。

SimpleRNN以外はゲート付きリカレントニューラルネットワークです。


**SimpleRNN**

**GRU**

**LSTM**

**ConvLSTM2D**

これらを実行してください。
この中でSimpleRNN、GRU、LSTMは同様のタスクに用いることができるため、精度の比較も行なってください。


Keras公式のサンプルコードを利用してください。



In [ ]:
# pip list
import tensorflow as tf
tf.test.gpu_device_name() 

'/device:GPU:0'

LSTMのサンプルコードを動かしてみる

[https://github.com/awslabs/keras-apache-mxnet/blob/master/examples/imdb_lstm.py](https://)

In [ ]:
'''-----翻訳-----
IMDBの感情分類タスクでLSTMモデルを学習します。
データセットが小さすぎるため、TF-IDF + LogRegのような単純で高速な手法と比較して、LSTMにはメリットがありません。
TF-IDF + LogRegのようなシンプルで高速な手法と比較して、LSTMには利点がありません。
# 注意点
- RNNは厄介です。バッチサイズの選択が重要。
損失とオプティマイザの選択が重要など。
収束しない設定もある。
- LSTMの学習中の損失減少パターンは、CNNやMLPなどとは
CNNやMLPなどで見られるものとはかなり異なる可能性があります。
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras import backend as K

max_features = 20000
# この語数以降のテキストをカット（max_featuresの上位の最もよく使われる語の中から
maxlen = 80
batch_size = 32

# 学習に使うデータの用意&確認
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('Build model...')

# モデルを定義
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 異なるオプティマイザーとオプティマイザーの設定を使用してみてください。
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,#epochを15から5へ変更
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Epoch 1/5
782/782 [==============================] - 479s 608ms/step - loss: 0.4463 - accuracy: 0.7876 - val_loss: 0.3746 - val_accuracy: 0.8353
Epoch 2/5
782/782 [==============================] - 469s 600ms/step - loss: 0.2574 - accuracy: 0.8966 - val_loss: 0.4137 - val_accuracy: 0.8282
Epoch 3/5
782/782 [==============================] - 468s 599ms/step - loss: 0.1668 - accuracy: 0.9380 - val_loss: 0.4727 - val_accuracy: 0.8272
Epoch 4/5
782/782 [==============================] - 463s 592ms/step - loss: 0.1139 - accuracy: 0.9592 - val_loss: 0.5306 - val_accuracy: 0.8196
Epoch 5/5
782/782 [==============================] - 26s 33ms/step - loss: 0.5934 - accuracy: 0.8196
Test score: 0.593350887298584
Test accuracy: 0.8196399807929993


ConvLSTM2Dのサンプルコードを動かしてみる

[https://github.com/awslabs/keras-apache-mxnet/blob/master/examples/conv_lstm.py](https://)

In [ ]:
!pip install tflearn

     |████████████████████████████████| 107 kB 5.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=75a99c9988084a6512900a9ca04f6a8a24822c1f6a358be3726f23dc9a0e0586
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
""" このスクリプトは、畳み込みLSTMネットワークの使用方法を示しています。
このネットワークは、動く四角形を含む人工的に生成されたムービーの次のフレームを予測するために使用されます。
次のフレームを予測します。
"""
import tensorflow as tf
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
# from keras.layers.normalization import BatchNormalization
# from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt
from tf.keras import backend as K

if K.backend() == 'mxnet':
    raise NotImplementedError("MXNet Backend: ConvLSTM2D Layer is not supported yet.")

# の形をしたムービーを入力とするレイヤーを作成します。
# (n_frames, width, height, channels)のムービーを入力とし、
# 同じ形の # ムービーを返すレイヤーを作ります。

seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 40, 40, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')

# 人工的なデータ生成。
# 内部に3〜7個の動く四角形がある動画を生成する。
# 升目の形状は1x1または2x2ピクセル。
# ピクセルの形をしており、時間の経過とともに直線的に動きます。
# ここでは、便宜上、幅と高さが大きいムービー（80x80）を作成し、最後に40x40のウィンドウを選択します。
# を作成し、最後に40x40のウィンドウを選択する。

def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # 3～7マスの移動マスを追加
        n = np.random.randint(3, 8)

        for j in range(n):
            # 初期位置
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # 動きの方向
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # サイズ
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1

                # ノイズを加えることで、よりロバストにする。
                # 考え方としては、推論中に
                # 推論中にピクセルの値が正確に1ではない場合は
                # ロバストになるようにネットワークを訓練し、それでもなお
                # 正方形に属するピクセルとみなします。
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # グラウンドトゥルースを1だけシフト
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # 40x40 windowにカットする
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

# ネットワークの学習
noisy_movies, shifted_movies = generate_movies(n_samples=1200)
seq.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10,
        epochs=150, validation_split=0.05)#epochを300→150へ変更

# 1つのムービーでネットワークをテスト
# 最初の7つのポジションをネットワークに入力してから
# 新しい位置を予測する
which = 1004
track = noisy_movies[which][:7, ::, ::, ::]

for j in range(16):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)

track2 = noisy_movies[which][::, ::, ::, ::]
for i in range(15):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('%i_animate.png' % (i + 1))


SimpleRNNを動かしてみる

In [30]:
# ライブラリのimport 
# ライブラリのimport
from keras.layers.recurrent import SimpleRNN
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.datasets import imdb
from keras import backend as K

# keras.layers.SimpleRNN(units, activation='tanh', use_bias=True, 
#                        kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
#                        bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, 
#                        bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
#                        recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, 
#                        return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)


In [32]:
#  imdbデータの読み込みと整形
max_features = 20000
maxlen = 80
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# 確認
x_train.shape, y_train.shape,x_test.shape, y_test.shape

((25000, 80), (25000,), (25000, 80), (25000,))

In [33]:
# モデルの定義
# Embeddingに関して：https://kento1109.hatenablog.com/entry/2017/12/02/114515
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [37]:
# コンパイルと学習
batch_size = 32

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=1,
    validation_data=(x_test, y_test)
)

782/782 [==============================] - 461s 586ms/step - loss: 0.4291 - accuracy: 0.7980 - val_loss: 0.3929 - val_accuracy: 0.8399


In [38]:
# 評価
score, acc = model.evaluate(
    x_test, 
    y_test,
    batch_size=batch_size
)

print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 27s 34ms/step - loss: 0.3929 - accuracy: 0.8399
Test score: 0.3928980529308319
Test accuracy: 0.8398799896240234


GRUを動かしてみる

In [39]:
from keras.layers.recurrent import GRU
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.datasets import imdb
from keras import backend as K

# imdbデータの読み込みと整形
max_features = 20000
maxlen = 80
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# 確認
x_train.shape, y_train.shape,x_test.shape, y_test.shape

# モデルの定義
# Embeddingに関して：https://kento1109.hatenablog.com/entry/2017/12/02/114515
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# コンパイルと学習
batch_size = 32
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=1,
    validation_data=(x_test, y_test)
)

# 評価
score, acc = model.evaluate(
    x_test, 
    y_test,
    batch_size=batch_size
)

print('Test score:', score)
print('Test accuracy:', acc)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
gru (GRU)                    (None, 128)               98688     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________
782/782 [==============================] - 26s 33ms/step - loss: 0.3647 - accuracy: 0.8424
Test score: 0.3647356629371643
Test accuracy: 0.8424000144004822


## 【問題3】他のクラスの説明

ドキュメントには他にも関連するクラスが記載されています。

それらがどういうものなのかを説明してください。

この中には実際に扱うことは少ないクラスも含まれています。


RNN

SimpleRNNCell

GRUCell

LSTMCell

StackedRNNCells

CuDNNGRU

CuDNNLSTM

### 回答


*   RNN　：再帰型ニューラルネットワーク（RNN）は、時系列や自然言語などのシーケンスデータのモデリングを強力に行うニューラルネットワークのクラスです。
*   SimpleRNNCell　：　SimpleRNNは前の時間ステップが次の時間ステップにフィードされる連結されたRNN
*   GRUCell　：　LSTMと同等の性能を持つが、LSTMより計算量が少なく高速な学習が可能（しかし表現能力はLSTMに劣るとの意見もあり）
*   LSTMCell　：　RNNで不可能であった長期的特徴の学習を可能にしたcell　計算コストが大きい
*   StackedRNNCells　：　RNNセルスタックの動作を単一セルのように見せるためのラッパーです。
*   CuDNNGRU　：　CuDNNを使った高速GRUの実装です。
*   CuDNNLSTM　：　CuDNNを用いた高速なLSTM実装です。







